<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/YouTubeToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

directory = '/content/drive/MyDrive'
files = os.listdir(directory)
print(directory)
print(files)

for i in files:
    try:
        os.remove(directory+'/'+i)
    except IsADirectoryError:
        pass


/content/drive/MyDrive
['Colab Notebooks', '.ipynb_checkpoints', 'a0xc4FdW6a4.mp3']


In [3]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 37.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=1f14e4b8ba37bea28b759699ad5e5707818cfdf0271c7842bba1080daeb6e0a6
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [4]:
pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.1 MB/s eta 0:00:00


In [5]:
pip install moviepy

In [6]:
pip install google-api-python-client

In [ ]:
import html
import os
import datetime
import csv
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
from pytubefix import YouTube
from moviepy.editor import *
import whisper

#-------↓パラメータ入力↓-------

developerKey = ""
channel_id = "UCgTDck8fPsLGQcJIYhdUCvw"

#-------↑パラメータ入力↑-------

youtube = build("youtube", "v3", developerKey=developerKey)

videos = [] #videoURLを格納する配列
dt_now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

def youtube_search(pagetoken, st, ed):

    search_response  = youtube.search().list(
        part = "snippet",
        channelId = channel_id,
        maxResults = 50,
        type = "video",
        publishedAfter = st, #'2013-01-01T00:00:00Z',
        publishedBefore = ed, #'2014-01-01T00:00:00Z',
        order = "date",
        pageToken = pagetoken
    ).execute()

    print(search_response["pageInfo"]["totalResults"])
    for search_result in search_response.get("items", []):

        if search_result["id"]["kind"] == "youtube#video":
            publishedAt = search_result["snippet"]["publishedAt"]
            title = html.unescape(search_result["snippet"]["title"])
            id = search_result["id"]["videoId"]
            url = 'https://www.youtube.com/watch?v=%s' % search_result["id"]["videoId"]

            videos.append([publishedAt, title, id, url])
            youtube_to_text(id, url)
    try:
        nextPagetoken = search_response["nextPageToken"]
        youtube_search(nextPagetoken, st, ed)
    except:
        return

def youtube_to_text(id, url):

    # YouTube動画をロード
    yt = YouTube(url)

    # 最高品質のオーディオストリームを選択
    audio_stream = yt.streams.filter(only_audio=True).first()

    # オーディオを一時ファイルとしてダウンロード
    temp_file = audio_stream.download()

    # 出力ファイル名
    audiofile = id + ".mp3"

    # MoviePyを使用してオーディオをMP3に変換
    audio_clip = AudioFileClip(temp_file)
    audio_clip.write_audiofile("/content/drive/MyDrive/" + audiofile, codec="libmp3lame")

    # 一時ファイルを削除
    os.remove(temp_file)

    # 出力ファイル名（適宜変更する）
    outputfile = id + ".txt"

    # 処理
    # model = whisper.load_model("smali", device="cpu")
    model = whisper.load_model("large", device="cuda")
    result = model.transcribe("/content/drive/MyDrive/" + audiofile, language="ja", verbose=True)

    # セグメントごとに改行してテキストを取得
    segments = result["segments"]
    transcript = "\n".join(segment["text"] for segment in segments)

    # txtへ書き出し
    with open("/content/drive/MyDrive/" + outputfile, "w", encoding='utf-8_sig') as f:
        f.write(transcript)

# 2023年1月1日から（ここを適宜変更）
dt = datetime.datetime(2023, 1, 1, 0, 0)
#
for i in range(1, 21):
    youtube_search('', dt.isoformat()+'Z', (dt + relativedelta(months=1)).isoformat()+'Z')
    dt = dt + relativedelta(months=1)

36
MoviePy - Writing audio in /content/drive/MyDrive/a0xc4FdW6a4.mp3


MoviePy - Done.


100%|█████████████████████████████████████| 2.88G/2.88G [00:35<00:00, 87.0MiB/s]
  checkpoint = torch.load(fp, map_location=device)



[00:00.560 --> 00:03.880] ルフィって全然海賊の船長っぽくない
[00:03.880 --> 00:07.000] 偉くなりたくねえ
[00:07.720 --> 00:10.000] 支配なんかしねえ
[00:11.040 --> 00:13.000] 助けた人たち
[00:13.000 --> 00:16.000] 同盟組んだ人たち
[00:16.720 --> 00:19.720] みんなのことを友達だと思ってる
[00:19.720 --> 00:22.520] みんな対等って感じですよね
[00:22.520 --> 00:29.440] でも麦わらの一味には船長風吹かして
[00:29.440 --> 00:32.660] みんなから慕われすぎるカリスマがいるんですよ
[00:32.660 --> 00:33.440] 誰かわかりますか?
[00:40.440 --> 00:42.340] ウソップ船長っぽくないか
[00:42.340 --> 00:45.240] いつも誰かに慕われ
[00:45.240 --> 00:46.940] 従えてますよね
[00:46.940 --> 00:50.660] 玉ねぎにんじんピーマン
[00:50.660 --> 00:52.440] お芋とカーシー
[00:53.280 --> 00:54.640] コンタッタ族など
[00:54.640 --> 00:57.340] 強いやつが引き勝ち
[00:58.660 --> 00:59.420] 仲間に
[00:59.420 --> 01:00.420] なっているとき
[01:00.420 --> 01:02.420] 俺が船長って言ってますしね
[01:02.420 --> 01:04.420] キャプテンウソップ
[01:06.420 --> 01:11.420] 実は船長っぽくない船長のルフィ
[01:11.420 --> 01:13.420] に対して
[01:13.420 --> 01:17.420] 船長っぽいただの船員ウソップ
[01:18.420 --> 01:22.420] 好物は肉と魚
[01:22.420 --> 01:25.420] わざわざ血統も描きました
[01:25.420 --> 01:27.420] ウソッ

MoviePy - Done.


...

[00:00.000 --> 00:02.440] こいつ、カラスが魚人だったら完全にハワイだわけですよ。
[00:02.700 --> 00:04.220] さすがにカラスが魚人だなと思ってきていないでしょ。
[00:04.340 --> 00:06.660] よく見てくださいよ。この人の肌はよ、人じゃないんだって。
[00:06.860 --> 00:09.600] 魚人系の肌の色じゃん。そもそもカラスって名前が露骨すぎて怪しい。
[00:09.800 --> 00:12.300] 鳥だと思い込ませに来てる。でもこの人鳥系の能力者ではないんですよ。
[00:12.540 --> 00:16.020] この人自体は飛びないし、今までの鳥系の能力者にこうやって鳥漏らす能力者はいなかった。
[00:16.360 --> 00:17.800] つまり、グリップバットみたいなことだと思います。
[00:18.140 --> 00:19.520] これもコウモリの能力者ではないんです。
[00:19.520 --> 00:22.000] カゲカゲの目でコウモリ出してるだけっていう、このパターン。
[00:22.320 --> 00:24.520] 水族館行っていろいろ考えたんですが、もう結論からいきますね。
[00:24.680 --> 00:26.120] えー、この人は人魚です。
[00:26.480 --> 00:27.640] 以前、タコの人魚出てきましたね。
[00:27.880 --> 00:28.720] 対して、イカの人魚。
[00:28.920 --> 00:30.720] 下半身が一切絶対に描かれてないの。
[00:31.380 --> 00:33.820] 納得ですよね。肌の色。納得ですよね。
[00:34.380 --> 00:35.400] イカスミ。納得ですよね。
[00:35.700 --> 00:37.120] タコとイカには、くちばしがあるんですよ。
[00:37.380 --> 00:38.680] こういうやつ。隠しませんね。
[00:39.400 --> 00:40.360] 絶対に描かれない下半身。
[00:40.640 --> 00:42.980] イカと同じ肌の色。イカスミ。隠しているくちばし。
[00:44.020 --> 0

MoviePy - Done.


  checkpoint = torch.load(fp, map_location=device)



[00:00.680 --> 00:02.820] 最近また懸賞金上がったんでしょ?
[00:03.000 --> 00:07.220] そうね、ナミ。9億3千万。ひどいことするわ。
[00:07.640 --> 00:09.000] 9億3千万?
[00:09.100 --> 00:09.500] そうね。
[00:10.020 --> 00:10.780] 似合わせ?
[00:10.900 --> 00:13.400] なにちょっと。ちょっとなに。
[00:18.940 --> 00:25.460] ロビンさん、小原唯一の生き残りって言われてますけど、サウル生きててよかったですね。
[00:25.460 --> 00:30.080] この世に生まれて一人ぼっちだということはないんだで。
[00:30.340 --> 00:31.980] いや、それあなた言われた側でしょ。
[00:32.040 --> 00:32.920] ひどいことするわ。
[00:33.840 --> 00:40.960] あの日あの場所で、世界政府に対して反逆の意思を持っていた男が4人いるんですよ。
[00:42.640 --> 00:49.100] 彼ら4人に対して、尾田先生ある要素を持たせています。なんだかわかりますか?
[00:49.560 --> 00:51.680] え、なにそれ。私のエロ同人誌の話?
[00:55.460 --> 00:59.240] 反逆の意思を持っていた4人。トランプ柄なんです。
[01:02.120 --> 01:03.440] まずクローバー博士。
[01:03.480 --> 01:04.660] クローバー?クリムソンかと思った。
[01:05.840 --> 01:10.760] ポーネグリフの元ネタは、おそらく大英博物館のロゼッタストーン。
[01:11.280 --> 01:12.600] ロゼッタっていうのは、
[01:12.600 --> 01:17.400] ロゼッタっていうのはクローバー模様のこと。
[01:18.480 --> 01:19.980] ここから来てると思います。
[01:20.820 --> 01:22.860] あなたに古代文字を教えてた人でしょ。
[01:25.460 --> 01:27.280] 最近登場したベガパ

chunk:  85%|████████▌ | 101563/119074 [02:10<00:23, 730.42it/s, now=None]

In [2]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/drive/MyDrive/download.zip /content/drive/MyDrive

  adding: content/drive/MyDrive/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled0.ipynb (deflated 58%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled1 (1).ipynb (deflated 87%)
  adding: content/drive/MyDrive/Colab Notebooks/Whisper YouTube in SRT.ipynb のコピー (deflated 60%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled2.ipynb (deflated 30%)
  adding: content/drive/MyDrive/Colab Notebooks/Whisper.ipynb のコピー (deflated 67%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled1.ipynb (deflated 66%)
  adding: content/drive/MyDrive/Colab Notebooks/WhisperOnGoogleColab.ipynb (deflated 66%)
  adding: content/drive/MyDrive/Colab Notebooks/YouTubeToText.ipynb (deflated 74%)
  adding: content/drive/MyDrive/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/a0xc4FdW6a4.mp3 (deflated 1%)
  adding: content/drive/MyDrive/a0xc4FdW6a4.txt (deflated 65%)
  adding: content/drive/MyDrive

In [3]:
# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/drive/MyDrive/download.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>